<a href="https://colab.research.google.com/github/limitlin840318/data-course-sample/blob/main/sample_content_based.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 基礎建設

In [1]:
import pandas as pd
import gzip, json

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

In [2]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np

## 載入資料

In [34]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz

--2022-01-02 18:04:36--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15499476 (15M) [application/octet-stream]
Saving to: ‘All_Beauty.csv.8’

All_Beauty.csv.8    100%[===================>]  14.78M  8.81MB/s    in 1.7s    

2022-01-02 18:04:38 (8.81 MB/s) - ‘All_Beauty.csv.8’ saved [15499476/15499476]

--2022-01-02 18:04:38--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10329961 (9.9M) [application/octet-stream]
Saving to: ‘meta_All_Beauty.json.gz.8’

meta_All_Beauty.jso 100%[===================>]   9.85M  6.58MB/s    in 1.5s   

In [35]:
#商品的基本資訊 metadata,購買商品的紀錄 ratings
metadata = getDF('/content/meta_All_Beauty.json.gz')
ratings = pd.read_csv('/content/All_Beauty.csv', names=['asin', 'reviewerID', 'overall', 'unixReviewTime'], header=None)

## 資料整理

In [36]:
ratings['DATE'] = pd.to_datetime(ratings['unixReviewTime'], unit='s')

In [37]:
# drop duplicates rows
metadata_clean = metadata.loc[metadata.astype(str).drop_duplicates().index]

In [38]:
# 商品資訊- select useful columns
metadata_clean = metadata_clean[['asin', 'brand', 'title', 'rank','description']]

In [39]:
metadata_clean['rank'] = metadata_clean['rank'].str.replace('&amp;', '&')
#metadata_clean['rank'].fillna('0', inplace = True)
# 商品資訊-買什麼種類
metadata_clean['categories'] = metadata_clean['rank'].str.split(' in ').str[1]
metadata_clean['categories'] = metadata_clean['categories'].str[:-2]

# 取出字串前段數字,取代掉千分位
metadata_clean['rank'] = metadata_clean['rank'].str.split(expand = True)[0].str.replace(',', '')
# 排名，轉換型別
metadata_clean['rank'] = pd.to_numeric(metadata_clean['rank'])

In [40]:
metadata_clean['categories'].unique()

array(['Beauty & Personal Care', nan, 'Grocery & Gourmet Food',
       'Health & Household', 'Sports & Outdoors', 'Toys & Games', 'Baby',
       'Clothing, Shoes & Jewelry', 'Home & Kitchen', 'Automotive',
       'Tools & Home Improvement'], dtype=object)

In [41]:
metadata_clean['title'] = metadata_clean['title'].str.replace('&amp;', '&')

In [42]:
spec_chars = ["!",'"',"#","%","&","'","(",")",
              "*","+",",","-",".","/",":",";","<",
              "=",">","?","@","[","\\","]","^","_",
              "`","{","|","}","~","–"]

In [43]:
metadata_clean['description'] = metadata_clean['description'].str[0].replace(np.nan, '')

In [44]:
for char in spec_chars:
    metadata_clean['description'] = metadata_clean['description'].str.replace(char, ' ')
metadata_clean['description'] = metadata_clean['description'].str.split().str.join(" ")

In [45]:
metadata_clean['desc'] = metadata_clean["title"] +' '+ metadata_clean["description"]
#metadata_clean['desc'] = metadata_clean['categories']  +' '+ metadata_clean["title"] +' '+ metadata_clean["description"]
metadata_clean['desc'] = metadata_clean['desc'].str.lower()

In [46]:
for char in spec_chars:
    metadata_clean['desc'] = metadata_clean['desc'].str.replace(char, ' ')
metadata_clean['desc'] = metadata_clean['desc'].str.split().str.join(" ")

In [47]:
#asin mean score
asin_rating = ratings.groupby('asin')[['overall']].mean().reset_index().rename(columns={"asin": "asin", "overall": "rating_mean"})

In [48]:
metadata_clean = pd.merge(metadata_clean,asin_rating , on="asin", how="left")

In [49]:
metadata_clean.head(10)

,asin,brand,title,rank,description,categories,desc,rating_mean
0,6546546450,idea village,Loud 'N Clear&trade; Personal Sound Amplifier,2938573.0,Loud N Clear Personal Sound Amplifier allows y...,Beauty & Personal Care,loud n clear trade personal sound amplifier lo...,2.5
1,7178680776,,No7 Lift & Luminate Triple Action Serum 50ml b...,872854.0,No7 Lift Luminate Triple Action Serum 50ml by ...,Beauty & Personal Care,no7 lift luminate triple action serum 50ml by ...,3.0
2,7250468162,No7,No7 Stay Perfect Foundation Cool Vanilla by No7,956696.0,No7 Stay Perfect Foundation now stays perfect ...,Beauty & Personal Care,no7 stay perfect foundation cool vanilla by no...,5.0
3,7367905066,,Wella Koleston Perfect Hair Colour 44/44 Mediu...,1870258.0,,Beauty & Personal Care,wella koleston perfect hair colour 44 44 mediu...,5.0
4,7414204790,Pirmal Healthcare,Lacto Calamine Skin Balance Oil control 120 ml...,67701.0,Lacto Calamine Skin Balance Daily Nourishing L...,Beauty & Personal Care,lacto calamine skin balance oil control 120 ml...,4.4
5,7535842801,Mary Kay,Mary Kay Satin Hands Hand Cream Travel MINI Si...,699317.0,Mary Kay Satin Hands Peach Hand Cream Travel S...,Beauty & Personal Care,mary kay satin hands hand cream travel mini si...,4.0
6,7538626107,,Unique Custom Cast Iron Liner Shader Tattoo Ma...,3206658.0,,Beauty & Personal Care,unique custom cast iron liner shader tattoo ma...,2.0
7,8279996397,Chango,Imagen Bendita Por Su Santidad Our Lady of Cha...,1741858.0,According to the legend in 1613 two brothers o...,Beauty & Personal Care,imagen bendita por su santidad our lady of cha...,5.0
8,8637910351,Srpska knjizevna zadruga,Mali princ,2136432.0,Novi prevod proslavljene knjige Zadruga objavl...,Beauty & Personal Care,mali princ novi prevod proslavljene knjige zad...,5.0
9,8867326759,Sunatoria,Moleskine Payne's Grey Card Wallet (Moleskine ...,1260339.0,Moleskine s Card Wallet in Payne s Grey is the...,Beauty & Personal Care,moleskine payne s grey card wallet moleskine n...,5.0


In [50]:
metadata_clean = metadata_clean[['asin','brand','rank','description','desc']]

## 資料切分

In [20]:
#依上次結果使用近三個月訓練資料
ratings_trainings = ratings[
    (ratings['DATE'] < '2018-09-01')
    ]

ratings_testings = ratings[
    (ratings['DATE'] >= '2018-09-01') & 
    (ratings['DATE'] <= '2018-09-30')
]

In [21]:
ratings_testings_by_user = ratings_testings.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin']].to_dict('records')
ratings_testings_by_user = { rating['reviewerID']: rating['asin'] for rating in ratings_testings_by_user }
users = list(ratings_testings_by_user.keys())

# Content-Based recommendation

一、Content-Based (description) + Rule-based (最近三個月內)

產生推薦
*   「描述」- tfidf矩陣
*   cosine_similarity，回傳k個最相近的物品 

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 計算商品用標題所表示的 tfidf 矩陣
df = metadata_clean.drop_duplicates('description')
tf = TfidfVectorizer(analyzer='word', stop_words='english')

In [23]:
tfidf_matrix = tf.fit_transform(df['description'])

In [24]:
# 計算商品間的相似程度
from sklearn.metrics.pairwise import cosine_similarity
similarity_matrix = cosine_similarity(tfidf_matrix)
mapping = pd.Series(df.index,index = df['description'])

In [25]:
# 每個商品回傳 k 個最相近的商品
def recommend_item(item_input, k=2):
    try:
        item_index = mapping[item_input]
        similarity_score = list(enumerate(similarity_matrix[item_index]))
        similarity_score = sorted(similarity_score, key=lambda x: x[1], reverse=True)
        similarity_score = similarity_score[:k]
        item_indices = [i[0] for i in similarity_score]
        return (df['asin'].iloc[item_indices].tolist())
    except:
        return []

# 利用使用者購買過的商品產生推薦
def recommend_items(items, k):
    res = []
    for d in items:
        res.extend(recommend_item(d, k))
    return res

In [ ]:
#for user in users:
  #print(ratings_trainings[ratings_trainings['reviewerID'] == user])
  #print(metadata[metadata['asin'].isin(ratings_trainings[ratings_trainings['reviewerID'] == user]['asin'].tolist())]['title'].tolist())
  #print(recommend_items(metadata[metadata['asin'].isin(ratings_trainings[ratings_trainings['reviewerID'] == user]['asin'].tolist())]['title'].tolist(), 2))

In [26]:
def recommender(training_data, users=[], k=10):
    '''
    * training_data: dataframe 輸入的訓練資料集（2018-09-01 以前資料）
    * users: [] 需要被推薦的使用者
    * k: int 每個使用者需要推薦的商品數
    * recommendations: dict
      {
          使用者一： [推薦商品一, 推薦商品二, ...],
          使用者二： [...], ...
      }
    '''
    recommendations = {}
    '''
    content-based
    '''
    ratings_trainings = training_data
    # 近三個月作為沒有購買記錄使用者的推薦商品
    ratings_trainings = ratings[
    (ratings['DATE'] < '2018-09-01') & (ratings['DATE'] >= '2018-06-01')
    ]

    purchased_user_list = ratings_trainings.reviewerID.tolist()
    top_ten = ratings_trainings.groupby('asin').count().sort_values('overall',ascending=False).head(k).index.tolist()

    for user in users:
      if user in purchased_user_list:
        recommendations[user] = recommend_items(metadata[metadata['asin'].isin(ratings_trainings[ratings_trainings['reviewerID'] == user]['asin'].tolist())]['description'].tolist(), k)
      else:
        recommendations[user] = top_ten
    
    return recommendations

ratings_by_user = recommender(ratings_trainings, users)

結果評估

In [27]:
def evaluate(ratings_testings_by_user={}, ratings_by_user={}, method=None):
    '''
    * ratings_testings_by_user: dict 真實被購買的商品資料（2018-09-01 以後資料）
    * ratings_by_user: dict 利用訓練資料學習的推薦商品
    * method: str
    * score: float
    '''
    total = 0
    for d in ratings_testings_by_user:
        if d in ratings_by_user:
            total += len(set(ratings_by_user[d]) & set(ratings_testings_by_user[d]))

    score = total / len(ratings_testings)
    return score

evaluate(ratings_testings_by_user, ratings_by_user)

0.13389830508474576

二、Content-Based (混合式欄位desc) 

產生推薦
*   「desc」- tfidf矩陣
*   cosine_similarity，回傳k個最相近的物品 

In [51]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 計算商品用標題所表示的 tfidf 矩陣
df = metadata_clean.drop_duplicates('desc')
tf = TfidfVectorizer(analyzer='word', stop_words='english')

In [52]:
tfidf_matrix = tf.fit_transform(df['desc'].values.astype('U'))

In [ ]:
# 計算商品間的相似程度
from sklearn.metrics.pairwise import cosine_similarity
similarity_matrix = cosine_similarity(tfidf_matrix)
mapping = pd.Series(df.index,index = df['desc'])

記憶體耗盡.....
